In [13]:
import pandas as pd
import numpy as np
import math
from datetime import datetime
import os

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report,
    roc_auc_score,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
)

from imblearn.ensemble import BalancedRandomForestClassifier
import joblib

---

## Train Driver-Order Accept Model

In [14]:
model_output_folder = "./out"
os.makedirs(model_output_folder, exist_ok=True)
model_path = "./Out/acceptance_model.pkl"

In [15]:
if os.path.exists(model_path):
    print(f"Model found at {model_path}. Loading model...")
    # with open(model_path, "rb") as f:
    #     model = pickle.load(f)
    model = joblib.load(model_path)
    print("Model loaded successfully.")
else:
    order_driver_data = pd.read_csv("./data/order_driver.csv")

    order_driver_data = order_driver_data.loc[
        (order_driver_data["status"] == 5) & (order_driver_data["outside"] == 0)
    ]
    print(order_driver_data.shape)
    print(order_driver_data["accept"].describe())

    # Define features & target variable
    # order_features = order_driver_data[["commission", "driver_distance", "hour", "weather_code", "work_time_minutes"]]
    order_features = order_driver_data[
        ["commission", "distance", "hour", "weather_code", "work_time_minutes"]
    ]
    print(order_features.head())
    acceptance_status = order_driver_data["accept"]
    # Define features & target variable
    # Train-test split
    features_train, features_test, target_train, target_test = train_test_split(
        order_features, acceptance_status, test_size=0.2, random_state=42
    )

    # Train model (BalancedRandomForest handles imbalance natively)
    model = BalancedRandomForestClassifier(random_state=42)
    model.fit(features_train, target_train)

    # Make predictions
    y_pred = model.predict(features_test)
    y_probs = model.predict_proba(features_test)[:, 1]

    # Evaluate model performance
    print("Precision:", precision_score(target_test, y_pred))
    print("Recall:", recall_score(target_test, y_pred))
    print("F1 Score:", f1_score(target_test, y_pred))
    print("Classification Report:\n", classification_report(target_test, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(target_test, y_pred))
    print("ROC AUC Score:", roc_auc_score(target_test, y_probs))
    # Save trained model (only the classifier, without SMOTE)
    joblib.dump(model, "./Out/acceptance_model.pkl")

Model found at ./Out/acceptance_model.pkl. Loading model...
Model loaded successfully.


---

## Get Weather Code

In [16]:
class WeatherService:
    def __init__(self, weather_csv_path: str):
        df = pd.read_csv(weather_csv_path)
        # Convert 'datetime' column to datetime objects and normalize to the hour start
        df["datetime"] = pd.to_datetime(df["datetime"]).dt.floor("H")
        # Set 'datetime' as index and convert 'weather_code' to a dictionary
        self.weather_data = df.set_index("datetime")["weather_code"].to_dict()

    def get_weather_code(self, dt) -> int:
        """Get weather code for the hour containing datetime dt"""
        hour_key = dt.replace(minute=0, second=0)
        # Default: 1 (sunny)
        return self.weather_data.get(hour_key, 1)

---

## Define the Order

In [17]:
class Order:
    """
    Represents a single customer order with details about pickup, dropoff, pricing,
    and calculated commission/revenue.
    """

    def __init__(
        self,
        order_id: int,
        datetime_str: str,
        pickup_area: int,
        dropoff_area: int,
        pickup_lat: float,
        pickup_lon: float,
        dropoff_lat: float,
        dropoff_lon: float,
        customer_price: float,
        commissionPercent: float,
    ):
        """
        Initializes an Order object.

        Args:
            order_id (int): Unique identifier for the order.
            datetime_str (str): Date and time of the order creation in '%Y-%m-%d %H:%M:%S.%f' format.
            pickup_area (int): Identifier for the pickup geographical area.
            dropoff_area (int): Identifier for the dropoff geographical area.
            pickup_lat (float): Latitude coordinate of the pickup location.
            pickup_lon (float): Longitude coordinate of the pickup location.
            dropoff_lat (float): Latitude coordinate of the dropoff location.
            dropoff_lon (float): Longitude coordinate of the dropoff location.
            customer_price (float): The total price paid by the customer for the order.
            commissionPercent (float): The percentage of the customer price taken as platform commission (e.g., 0.20 for 20%).
        """
        self.order_id = order_id
        # Convert datetime string to a datetime object for easier manipulation
        self.datetime = datetime.strptime(datetime_str, "%Y-%m-%d %H:%M:%S.%f")
        self.pickup_area = pickup_area
        self.dropoff_area = dropoff_area
        self.pickup_lat = pickup_lat
        self.pickup_lon = pickup_lon
        self.dropoff_lat = dropoff_lat
        self.dropoff_lon = dropoff_lon
        self.customer_price = customer_price
        self.commissionPercent = commissionPercent

        # These calculations were previously in __post_init__
        # Calculate the driver's earnings from the order
        self.driver_commission = self.customer_price * (1 - self.commissionPercent)
        # Calculate the platform's revenue from the order
        self.platform_revenue = self.customer_price * self.commissionPercent
        # Extract the hour of the day when the order was placed (0-23)
        self.hour_of_day = self.datetime.hour

---

## Define the Driver

In [18]:
class Driver:
    def __init__(
        self,
        driver_id: int,
        current_lat: float,
        current_lon: float,
        current_area: int,
        work_time_minutes: float,
        available: bool = True,
        accepted_order: bool = False,
    ):
        self.driver_id = driver_id
        self.current_lat = current_lat
        self.current_lon = current_lon
        self.current_area = current_area
        self.work_time_minutes = work_time_minutes
        self.available = available
        self.accepted_order = accepted_order
        self.model = None  # Model still needs to be set externally

        print(
            f"Driver {self.driver_id} is initialized with location ({self.current_lat}, {self.current_lon})"
        )

    def distance_to(self, order: Order) -> float:
        """Calculate Euclidean distance in kilometers (approx)."""
        return (
            math.sqrt(
                (self.current_lat - order.dropoff_lat) ** 2
                + (self.current_lon - order.dropoff_lon) ** 2
            )
            * 111
        ) * 1000

    def calculate_accept_prob(self, order: Order, weather_code: int) -> float:
        """Predict acceptance probability using logistic regression."""
        if self.model is None:
            raise ValueError(
                "Driver model not initialized! Must be set in DeliverySimulator."
            )

        features = {
            "commission": [order.driver_commission],
            "distance": [self.distance_to(order)],
            "hour": [order.hour_of_day],
            "weather_code": [weather_code],
            "work_time_minutes": [self.work_time_minutes],
        }
        print("Features input to the model for prediction:")
        print(features)
        return self.model.predict_proba(pd.DataFrame(features))[0][1]

    def decide_acceptance(
        self,
        order: Order,
        weather_code: int,
        threshold: float = np.random.random(),
    ) -> bool:
        """Make acceptance decision based on probability."""
        if not self.available:
            return False

        # random_value = np.random.random()
        random_value = threshold
        # print(f"Random Value: {random_value:.2f}")
        prob = self.calculate_accept_prob(order, weather_code)

        accepted = bool(random_value < prob)
        if accepted:
            print(
                f"Driver {self.driver_id} accept the order with probability of {prob} and threshold {threshold}"
            )
            self.accepted_order = True
        else:
            print(
                f"Driver {self.driver_id} did not accept the order with probability of {prob} and threshold {threshold}"
            )
        return accepted

    def update_location(self, order: Order):
        """Update location only if the driver has taken an order."""
        if self.accepted_order:
            self.current_lat = order.dropoff_lat
            self.current_lon = order.dropoff_lon
            self.current_area = order.dropoff_area
            print(
                f"Driver {self.driver_id} location moves to ({self.current_lat}, {self.current_lon})"
            )
        else:
            print(f"Driver {self.driver_id} keeps the same location")

---

## Test Order and Driver

In [19]:
test_driver = Driver(
    driver_id=999,
    current_lat=34.0,
    current_lon=-118.0,
    current_area=100,
    work_time_minutes=300,
    available=True,
)

test_order = Order(
    order_id=1,
    datetime_str="2023-01-15 10:30:00.000000",
    pickup_area=101,
    dropoff_area=202,
    pickup_lat=34.05,
    pickup_lon=-118.05,
    dropoff_lat=34.1,
    dropoff_lon=-118.1,
    customer_price=100.0,
    commissionPercent=0.20,
)

test_weather_code = 1
test_driver.model = model

Driver 999 is initialized with location (34.0, -118.0)


In [22]:
# manully set threshold
decide_pred = test_driver.decide_acceptance(
    test_order, test_weather_code, threshold=0.5
)

# random threshold
decide_pred = test_driver.decide_acceptance(test_order, test_weather_code)

Features input to the model for prediction:
{'commission': [80.0], 'distance': [15697.770542341023], 'hour': [10], 'weather_code': [1], 'work_time_minutes': [300]}
Driver 999 did not accept the order with probability of 0.41 and threshold 0.5
Features input to the model for prediction:
{'commission': [80.0], 'distance': [15697.770542341023], 'hour': [10], 'weather_code': [1], 'work_time_minutes': [300]}
Driver 999 accept the order with probability of 0.41 and threshold 0.3653670961416522


In [12]:
location_pred = test_driver.update_location(test_order)

Driver 999 keeps the same location


---

# Define Ride Hail Env

In [ ]:
import gym
from gym import spaces
from collections import defaultdict


class DeliveryEnv(gym.Env):
    def __init__(
        self,
        orders: list,
        driver_data: pd.DataFrame,
        schedule_data: pd.DataFrame,
        weather_service: WeatherService,
    ):
        super(DeliveryEnv, self).__init__()

        # self.simulator = DeliverySimulator(
        #     orders, driver_data, schedule_data, weather_service
        # )
        self.orders = sorted(orders, key=lambda o: o.datetime)
        self.weather = weather_service
        self.driver_schedule = self._load_driver_schedule(schedule_data)
        self.driver_attempts = self._load_driver_attempts(driver_data)
        self.drivers_by_id = {}  # Cache all drivers by ID
        self.area_drivers = self._group_drivers_by_area()
        # Define action space (continuous commission rate between 0 and 1)
        self.action_space = spaces.Box(low=0.0, high=1.0, shape=(1,), dtype=np.float32)

        # Define state space (order + specific driver attributes)
        self.observation_space = spaces.Dict(
            {
                "customer_price": spaces.Box(
                    low=0.0, high=1.0, shape=(1,), dtype=np.float32
                ),
                "pickup_area": spaces.Discrete(501),
                "dropoff_area": spaces.Discrete(501),
                "hour_of_day": spaces.Discrete(24),
                "day_of_week": spaces.Discrete(7),
                "weather": spaces.Discrete(4),
                "driver_area": spaces.Discrete(501),
                "working_status": spaces.Discrete(2),
            }
        )

        self.orders_by_day = defaultdict(list)
        for order in self.simulator.orders:
            order_day = order.datetime.date()
            self.orders_by_day[order_day].append(order)

        # Tracking variables
        self.assigned_order = 0  # Tracks # of unassigned orders
        self.current_day_index = 0  # Tracks training epoch (day index)
        self.current_order_index = 0  # Tracks current order within the day
        self.current_driver_index = 0
        self.current_day = None  # Current date being trained
        self.updated_drivers = (
            set()
        )  # Track drivers who have accepted at least one order
        self.next_order = False
        self.episode_rewards = 0
        self.episode_steps = 0
        self.total_driver_commission = 0.0
        self.max_steps = 30000

    def _load_driver_schedule(self, schedule_data: pd.DataFrame):
        """Loads driver work schedules from a CSV file into a dictionary."""
        schedule = defaultdict(set)
        for _, row in schedule_data.iterrows():
            driver_id = row["driver_id"]
            date = row["date"]
            hour = row["hour"]
            schedule[(driver_id, date)].add(hour)
        return schedule

    def _load_driver_attempts(self, driver_data: pd.DataFrame):
        """Loads driver assignment attempts, tracking all instances a driver receives an order."""
        attempts = defaultdict(list)
        for _, row in driver_data.iterrows():
            order_id = row["order_id"]
            driver_id = row["driver_id"]
            datetime = row["datetime"]
            lat, lon, area = row["driver_lat"], row["driver_lon"], row["driver_area"]
            work_time_minutes = row["work_time_minutes"]
            attempts[order_id].append(
                (driver_id, datetime, lat, lon, area, work_time_minutes)
            )
        return attempts

    def _group_drivers_by_area(self):
        """Groups drivers by their current area for efficient order assignment."""
        area_drivers = defaultdict(list)
        for order_id, driver_attempts in self.driver_attempts.items():
            for (
                driver_id,
                datetime,
                lat,
                lon,
                area,
                work_time_minutes,
            ) in driver_attempts:
                if driver_id not in self.drivers_by_id:
                    driver = Driver(
                        driver_id=driver_id,
                        current_lat=lat,
                        current_lon=lon,
                        current_area=area,
                        work_time_minutes=work_time_minutes,
                    )
                    # driver.model = DeliverySimulator.shared_model
                    self.drivers_by_id[driver_id] = driver
                    area_drivers[area].append(driver)
        return area_drivers